In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd
from joblib import Parallel, delayed

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [3]:
def add_ad_to_car_dict(ad):
    try:
        car_url = ad
        response = requests.get(car_url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
    except:
        return
    try:
        json_data = json.loads(
            page.find('script', type="application/ld+json").string)
    except:
        json_data = None
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_url = catalog_url.get('href')
    except:
        return
    response_catalog = requests.get(catalog_url)
    response_catalog.encoding = 'utf-8'
    page_catalog = BeautifulSoup(response_catalog.text, 'html.parser')
    try:
        json_data_catalog = json.loads(page_catalog.find(
            'script', type="application/json", id='initial-state').string)
    except:
        json_data_catalog = None
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        json_data_equip = None
    try:
        json_data['description'] = json_data['description'].replace('\n', ' ')
        json_data['description'] = re.sub('\W+', ' ', json_data['description'])
    except:
        json_data['description'] = ''
    try:
        options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
        options_list = [key for key in list(
            options_dict.keys()) if options_dict[key] == 1]
    except:
        options_list = []
    car_dict.update({ad: {}})
    try:
        car_dict[ad]['car_url'] = json_data['offers']['url']
    except:
        car_dict[ad]['car_url'] = None
    try:
        car_dict[ad]['bodyType'] = json_data['bodyType']
    except:
        car_dict[ad]['bodyType'] = None
    try:
        car_dict[ad]['brand'] = json_data['brand']
    except:
        car_dict[ad]['brand'] = None
    try:
        car_dict[ad]['color'] = json_data['color']
    except:
        car_dict[ad]['color'] = None
    try:
        car_dict[ad]['complectation_dict'] = options_list
    except:
        car_dict[ad]['complectation_dict'] = None
    try:
        car_dict[ad]['description'] = json_data['description']
    except:
        car_dict[ad]['description'] = None
    try:
        car_dict[ad]['engineDisplacement'] = json_data['vehicleEngine']['engineDisplacement']
    except:
        car_dict[ad]['engineDisplacement'] = None
    try:
        car_dict[ad]['enginePower'] = json_data['vehicleEngine']['enginePower']
    except:
        car_dict[ad]['enginePower'] = None
    try:
        car_dict[ad]['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        car_dict[ad]['equipment_dict'] = None
    try:
        car_dict[ad]['fuelType'] = json_data['fuelType']
    except:
        car_dict[ad]['fuelType'] = None
    try:
        car_dict[ad]['image'] = json_data['image']
    except:
        car_dict[ad]['image'] = None
    try:
        car_dict[ad]['mileage'] = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['mileage'] = None
    try:
        car_dict[ad]['modelDate'] = json_data['modelDate']
    except:
        car_dict[ad]['modelDate'] = None
    try:
        car_dict[ad]['model_name'] = page.find_all(
            'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        car_dict[ad]['model_name'] = None
    try:
        car_dict[ad]['name'] = json_data['name']
    except:
        car_dict[ad]['name'] = None
    try:
        car_dict[ad]['numberOfDoors'] = json_data['numberOfDoors']
    except:
        car_dict[ad]['numberOfDoors'] = None
    try:
        car_dict[ad]['parsing_unixtime'] = int(time.time())
    except:
        car_dict[ad]['parsing_unixtime'] = None
    try:
        car_dict[ad]['price'] = json_data['offers']['price']
    except:
        car_dict[ad]['price'] = None
    try:
        car_dict[ad]['priceCurrency'] = json_data['offers']['priceCurrency']
    except:
        car_dict[ad]['priceCurrency'] = None
    try:
        car_dict[ad]['productionDate'] = json_data['productionDate']
    except:
        car_dict[ad]['productionDate'] = None
    try:
        car_dict[ad]['sell_id'] = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
    except:
        car_dict[ad]['sell_id'] = None
    try:
        car_dict[ad]['views'] = page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        car_dict[ad]['views'] = None
    try:
        car_dict[ad]['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text
    except:
        car_dict[ad]['date_added'] = None
    try:
        car_dict[ad]['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        car_dict[ad]['super_gen'] = None
    try:
        car_dict[ad]['vehicleConfiguration'] = json_data['vehicleConfiguration']
    except:
        car_dict[ad]['vehicleConfiguration'] = None
    try:
        car_dict[ad]['vehicleTransmission'] = json_data['vehicleTransmission']
    except:
        car_dict[ad]['vehicleTransmission'] = None
    try:
        car_dict[ad]['Владельцы'] = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['Владельцы'] = None
    try:
        car_dict[ad]['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        car_dict[ad]['Владение'] = None
    try:
        car_dict[ad]['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        car_dict[ad]['ПТС'] = None
    try:
        car_dict[ad]['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        car_dict[ad]['Привод'] = None
    try:
        car_dict[ad]['Руль'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        car_dict[ad]['Руль'] = None
    try:
        car_dict[ad]['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        car_dict[ad]['Состояние'] = None
    try:
        car_dict[ad]['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        car_dict[ad]['Таможня'] = None
    try:
        car_dict[ad]['region'] = page.find('div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        car_dict[ad]['region'] = None

df_ads = pd.read_csv('Project_6.csv', sep=',', header=None)
all_ads = df_ads[0].tolist()
car_dict = {}
i = 230000
for ad in all_ads[230000:]:
    print(i)
    add_ad_to_car_dict(ad)
    i += 1
    if i % 10 == 0:
        print(f'Completed {i} of total {len(all_ads)}')
    else:
        continue
df_10 = pd.DataFrame.from_dict(car_dict, orient='index')
df_10.to_csv('Final_df_10.csv')

230000
230001
230002
230003
230004
230005
230006
230007
230008
230009
Completed 230010 of total 269381
230010
230011
230012
230013
230014
230015
230016
230017
230018
230019
Completed 230020 of total 269381
230020
230021
230022
230023
230024
230025
230026
230027
230028
230029
Completed 230030 of total 269381
230030
230031
230032
230033
230034
230035
230036
230037
230038
230039
Completed 230040 of total 269381
230040
230041
230042
230043
230044
230045
230046
230047
230048
230049
Completed 230050 of total 269381
230050
230051
230052
230053
230054
230055
230056
230057
230058
230059
Completed 230060 of total 269381
230060
230061
230062
230063
230064
230065
230066
230067
230068
230069
Completed 230070 of total 269381
230070
230071
230072
230073
230074
230075
230076
230077
230078
230079
Completed 230080 of total 269381
230080
230081
230082
230083
230084
230085
230086
230087
230088
230089
Completed 230090 of total 269381
230090
230091
230092
230093
230094
230095
230096
230097
230098
230099
Com

230798
230799
Completed 230800 of total 269381
230800
230801
230802
230803
230804
230805
230806
230807
230808
230809
Completed 230810 of total 269381
230810
230811
230812
230813
230814
230815
230816
230817
230818
230819
Completed 230820 of total 269381
230820
230821
230822
230823
230824
230825
230826
230827
230828
230829
Completed 230830 of total 269381
230830
230831
230832
230833
230834
230835
230836
230837
230838
230839
Completed 230840 of total 269381
230840
230841
230842
230843
230844
230845
230846
230847
230848
230849
Completed 230850 of total 269381
230850
230851
230852
230853
230854
230855
230856
230857
230858
230859
Completed 230860 of total 269381
230860
230861
230862
230863
230864
230865
230866
230867
230868
230869
Completed 230870 of total 269381
230870
230871
230872
230873
230874
230875
230876
230877
230878
230879
Completed 230880 of total 269381
230880
230881
230882
230883
230884
230885
230886
230887
230888
230889
Completed 230890 of total 269381
230890
230891
230892
23089

231592
231593
231594
231595
231596
231597
231598
231599
Completed 231600 of total 269381
231600
231601
231602
231603
231604
231605
231606
231607
231608
231609
Completed 231610 of total 269381
231610
231611
231612
231613
231614
231615
231616
231617
231618
231619
Completed 231620 of total 269381
231620
231621
231622
231623
231624
231625
231626
231627
231628
231629
Completed 231630 of total 269381
231630
231631
231632
231633
231634
231635
231636
231637
231638
231639
Completed 231640 of total 269381
231640
231641
231642
231643
231644
231645
231646
231647
231648
231649
Completed 231650 of total 269381
231650
231651
231652
231653
231654
231655
231656
231657
231658
231659
Completed 231660 of total 269381
231660
231661
231662
231663
231664
231665
231666
231667
231668
231669
Completed 231670 of total 269381
231670
231671
231672
231673
231674
231675
231676
231677
231678
231679
Completed 231680 of total 269381
231680
231681
231682
231683
231684
231685
231686
231687
231688
231689
Completed 231690 

Completed 232390 of total 269381
232390
232391
232392
232393
232394
232395
232396
232397
232398
232399
Completed 232400 of total 269381
232400
232401
232402
232403
232404
232405
232406
232407
232408
232409
Completed 232410 of total 269381
232410
232411
232412
232413
232414
232415
232416
232417
232418
232419
Completed 232420 of total 269381
232420
232421
232422
232423
232424
232425
232426
232427
232428
232429
Completed 232430 of total 269381
232430
232431
232432
232433
232434
232435
232436
232437
232438
232439
Completed 232440 of total 269381
232440
232441
232442
232443
232444
232445
232446
232447
232448
232449
Completed 232450 of total 269381
232450
232451
232452
232453
232454
232455
232456
232457
232458
232459
Completed 232460 of total 269381
232460
232461
232462
232463
232464
232465
232466
232467
232468
232469
Completed 232470 of total 269381
232470
232471
232472
232473
232474
232475
232476
232477
232478
232479
Completed 232480 of total 269381
232480
232481
232482
232483
232484
23248

233184
233185
233186
233187
233188
233189
Completed 233190 of total 269381
233190
233191
233192
233193
233194
233195
233196
233197
233198
233199
Completed 233200 of total 269381
233200
233201
233202
233203
233204
233205
233206
233207
233208
233209
Completed 233210 of total 269381
233210
233211
233212
233213
233214
233215
233216
233217
233218
233219
Completed 233220 of total 269381
233220
233221
233222
233223
233224
233225
233226
233227
233228
233229
Completed 233230 of total 269381
233230
233231
233232
233233
233234
233235
233236
233237
233238
233239
Completed 233240 of total 269381
233240
233241
233242
233243
233244
233245
233246
233247
233248
233249
Completed 233250 of total 269381
233250
233251
233252
233253
233254
233255
233256
233257
233258
233259
Completed 233260 of total 269381
233260
233261
233262
233263
233264
233265
233266
233267
233268
233269
Completed 233270 of total 269381
233270
233271
233272
233273
233274
233275
233276
233277
233278
233279
Completed 233280 of total 26938

...

265849
Completed 265850 of total 269381
265850
265851
265852
265853
265854
265855
265856
265857
265858
265859
Completed 265860 of total 269381
265860
265861
265862
265863
265864
265865
265866
265867
265868
265869
Completed 265870 of total 269381
265870
265871
265872
265873
265874
265875
265876
265877
265878
265879
Completed 265880 of total 269381
265880
265881
265882
265883
265884
265885
265886
265887
265888
265889
Completed 265890 of total 269381
265890
265891
265892
265893
265894
265895
265896
265897
265898
265899
Completed 265900 of total 269381
265900
265901
265902
265903
265904
265905
265906
265907
265908
265909
Completed 265910 of total 269381
265910
265911
265912
265913
265914
265915
265916
265917
265918
265919
Completed 265920 of total 269381
265920
265921
265922
265923
265924
265925
265926
265927
265928
265929
Completed 265930 of total 269381
265930
265931
265932
265933
265934
265935
265936
265937
265938
265939
Completed 265940 of total 269381
265940
265941
265942
265943
26594

266643
266644
266645
266646
266647
266648
266649
Completed 266650 of total 269381
266650
266651
266652
266653
266654
266655
266656
266657
266658
266659
Completed 266660 of total 269381
266660
266661
266662
266663
266664
266665
266666
266667
266668
266669
Completed 266670 of total 269381
266670
266671
266672
266673
266674
266675
266676
266677
266678
266679
Completed 266680 of total 269381
266680
266681
266682
266683
266684
266685
266686
266687
266688
266689
Completed 266690 of total 269381
266690
266691
266692
266693
266694
266695
266696
266697
266698
266699
Completed 266700 of total 269381
266700
266701
266702
266703
266704
266705
266706
266707
266708
266709
Completed 266710 of total 269381
266710
266711
266712
266713
266714
266715
266716
266717
266718
266719
Completed 266720 of total 269381
266720
266721
266722
266723
266724
266725
266726
266727
266728
266729
Completed 266730 of total 269381
266730
266731
266732
266733
266734
266735
266736
266737
266738
266739
Completed 266740 of tota

267441
267442
267443
267444
267445
267446
267447
267448
267449
Completed 267450 of total 269381
267450
267451
267452
267453
267454
267455
267456
267457
267458
267459
Completed 267460 of total 269381
267460
267461
267462
267463
267464
267465
267466
267467
267468
267469
Completed 267470 of total 269381
267470
267471
267472
267473
267474
267475
267476
267477
267478
267479
Completed 267480 of total 269381
267480
267481
267482
267483
267484
267485
267486
267487
267488
267489
Completed 267490 of total 269381
267490
267491
267492
267493
267494
267495
267496
267497
267498
267499
Completed 267500 of total 269381
267500
267501
267502
267503
267504
267505
267506
267507
267508
267509
Completed 267510 of total 269381
267510
267511
267512
267513
267514
267515
267516
267517
267518
267519
Completed 267520 of total 269381
267520
267521
267522
267523
267524
267525
267526
267527
267528
267529
Completed 267530 of total 269381
267530
267531
267532
267533
267534
267535
267536
267537
267538
267539
Completed 

268239
Completed 268240 of total 269381
268240
268241
268242
268243
268244
268245
268246
268247
268248
268249
Completed 268250 of total 269381
268250
268251
268252
268253
268254
268255
268256
268257
268258
268259
Completed 268260 of total 269381
268260
268261
268262
268263
268264
268265
268266
268267
268268
268269
Completed 268270 of total 269381
268270
268271
268272
268273
268274
268275
268276
268277
268278
268279
Completed 268280 of total 269381
268280
268281
268282
268283
268284
268285
268286
268287
268288
268289
Completed 268290 of total 269381
268290
268291
268292
268293
268294
268295
268296
268297
268298
268299
Completed 268300 of total 269381
268300
268301
268302
268303
268304
268305
268306
268307
268308
268309
Completed 268310 of total 269381
268310
268311
268312
268313
268314
268315
268316
268317
268318
268319
Completed 268320 of total 269381
268320
268321
268322
268323
268324
268325
268326
268327
268328
268329
Completed 268330 of total 269381
268330
268331
268332
268333
26833

269033
269034
269035
269036
269037
269038
269039
Completed 269040 of total 269381
269040
269041
269042
269043
269044
269045
269046
269047
269048
269049
Completed 269050 of total 269381
269050
269051
269052
269053
269054
269055
269056
269057
269058
269059
Completed 269060 of total 269381
269060
269061
269062
269063
269064
269065
269066
269067
269068
269069
Completed 269070 of total 269381
269070
269071
269072
269073
269074
269075
269076
269077
269078
269079
Completed 269080 of total 269381
269080
269081
269082
269083
269084
269085
269086
269087
269088
269089
Completed 269090 of total 269381
269090
269091
269092
269093
269094
269095
269096
269097
269098
269099
Completed 269100 of total 269381
269100
269101
269102
269103
269104
269105
269106
269107
269108
269109
Completed 269110 of total 269381
269110
269111
269112
269113
269114
269115
269116
269117
269118
269119
Completed 269120 of total 269381
269120
269121
269122
269123
269124
269125
269126
269127
269128
269129
Completed 269130 of tota